# Python-Powered Algorithmic Trading: From Theory to Practice

## Introduction


### Algorithmic Trading

Algorithmic trading uses computer algorithms to automatically execute trades based on predefined criteria, such as price, volume, or timing.

It aims to optimize trading strategies, minimize human error, and capitalize on market opportunities quickly.

By analyzing vast amounts of data and executing trades at high speeds, algo trading enhances efficiency and can lead to more precise and profitable trading decisions.


## Installation

### Configurations

In [1]:
# Set Configurations

# Matplotlib configuration
%matplotlib inline

# Suppress future warning messages
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

### TA-Lib C Module

[TA-Lib](https://ta-lib.org/) (Technical Analysis Library) is widely used by developers in finance and trading for performing technical analysis on market data.

- Provides over 200+ indicators such as ADX, MACD, RSI, Stochastic, Bollinger Bands, and more.
- Supports candlestick pattern recognition for identifying trading signals.
- Released in 2001, TA-Lib implements well-established algorithms that are still popular over 20 years later.
- Provides Open-source API for Python, but requires installation of a C module dependency for proper functionality.

Installation:
To install the required C module for TA-Lib, specific commands are needed.
For installation in Google Colab, refer to a [Stackoverflow answer](https://stackoverflow.com/questions/49648391/how-to-install-ta-lib-in-google-colab) that provides step-by-step guidance.

In [2]:
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb

Selecting previously unselected package libta-lib0.
(Reading database ... 126371 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.8) ...
/sbin/ldconfig.real: /usr/local/lib/libur_adapter_level_zero.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_loader.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_level_zero_v2.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm.so.1 is not a symbol

### TA-Lib Python Package

The original Python bindings for TA-Lib were created using **SWIG** (Simplified Wrapper and Interface Generator). However, these bindings can be challenging to install and are not as optimized in terms of performance as they could be.

To address these limitations, we will use a different [Python package](https://github.com/TA-Lib/ta-lib-python) that serves as a **Cython-based wrapper** for TA-Lib. Cython provides a more streamlined and efficient way to interface with the underlying C library, making it easier to install and faster in execution compared to the SWIG-based implementation.

This alternative package ensures better performance and ease of use, making it a preferred choice for developers working with TA-Lib in Python.

In [3]:
!pip install TA-Lib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 40.4 MB/s eta 0:00:00


### Yfinance
[yfinance](https://github.com/ranaroussi/yfinance) provides a Pythonic and efficient way to download market data, making it easy to access historical and real-time financial information.

In [4]:
!pip install yfinance

### Backtrader
[Backtrader](https://github.com/mementum/backtrader) is a popular open-source Python library designed for backtesting trading strategies.

It allows traders and developers to test and analyze their trading ideas using historical data before applying them in real-world markets.

In [5]:
!pip install backtrader
!pip install backtrader[plotting]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 5.6 MB/s eta 0:00:00


### Import Packages

In [6]:
import talib
import yfinance
import backtrader as bt
import pytz
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

## Get Instrument Data

In [ ]:
def get_instrument_data(symbol: str, period="1mo", interval="1d") -> pd.DataFrame:
    """
    Fetches historical instrument data for a given symbol within a specified period.

    Args:
        symbol (str): The instrument ticker symbol (e.g., 'AAPL' for Apple).
        period (str): Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        interval (str): Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                        Intraday data cannot extend last 60 days

    Returns:
        pd.DataFrame: DataFrame with columns: 'Open', 'Close', 'High', 'Low' and 'Volume'.
        
    Raises:
        ValueError: If symbol is empty or invalid parameters are provided
        Exception: For network or data retrieval errors
    """
    # Input validation
    if not symbol or not isinstance(symbol, str):
        raise ValueError("Symbol must be a non-empty string")
    
    valid_periods = ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
    valid_intervals = ['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo']
    
    if period not in valid_periods:
        raise ValueError(f"Invalid period. Must be one of: {valid_periods}")
    
    if interval not in valid_intervals:
        raise ValueError(f"Invalid interval. Must be one of: {valid_intervals}")
    
    try:
        # Create a Ticker object for the given symbol
        ticker = yfinance.Ticker(symbol)

        # Fetch historical data
        instrument_data_df = ticker.history(period, interval, auto_adjust=False,
                                            rounding=True, actions=False)
        
        # Validate that data was retrieved
        if instrument_data_df.empty:
            raise ValueError(f"No data found for symbol {symbol} with period {period} and interval {interval}")
        
        # Check for required columns
        required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        missing_columns = [col for col in required_columns if col not in instrument_data_df.columns]
        if missing_columns:
            raise ValueError(f"Missing required columns: {missing_columns}")

        return instrument_data_df
        
    except Exception as e:
        print(f"Error fetching data for {symbol}: {str(e)}")
        raise

In [20]:
data = get_instrument_data("TMC", "1mo", "15m")

### Ticker Symbols


| Category     | Instrument   | Symbol      |
|--------------|--------------|-------------|
| Stock        | RELIANCE     | RELIANCE.NS |
|              | HDFC         | HDFCBANK.NS |
|              |              |             |
| Index        | Nifty50      | ^NSEI       |
|              | Bank Nifty   | ^NSEBANK    |
|              |              |             |
| Crypto       | Bitcoin      | BTC-INR     |
|              | Ethereum     | ETH-INR     |
|              |              |             |


You can search for the symbol of any instrument on [Yahoo's website](https://finance.yahoo.com/lookup/).


### Preview Stock Data

In [21]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2025-08-21 14:45:00-04:00,4.635,4.7283,4.6300,4.7198,4.7198,271024
2025-08-21 15:00:00-04:00,4.710,4.7699,4.7000,4.7300,4.7300,384384
2025-08-21 15:15:00-04:00,4.735,4.7700,4.7000,4.7450,4.7450,328675
2025-08-21 15:30:00-04:00,4.750,4.9450,4.7346,4.8750,4.8750,1145110
2025-08-21 15:45:00-04:00,4.870,5.0150,4.8550,4.9600,4.9600,3465632


## Visualize Stock Data

### Line Chart
A line chart displays the stock's closing price over time, offering a clear view of overall trends. It is a simple yet effective way to track price movements and identify upward or downward trends.

In [12]:
def display_line_chart(data):
  fig = px.line(data, x=data.index, y="Close")
  fig.show()

In [22]:
display_line_chart(data)

### Candlestick Chart
A candlestick chart visually represents the stock's open, high, low, and close prices for each time period. It is commonly used in technical analysis to detect patterns and trends in market behavior.

In [14]:
def display_candlestick_chart(data):
  candlestick_data = go.Candlestick(x=data.index,
                                  open=data['Open'],
                                  high=data['High'],
                                  low=data['Low'],
                                  close=data['Close'])
  fig = go.Figure(data=[candlestick_data])
  fig.show()

In [23]:
display_candlestick_chart(data)

## Instantiating Cerebro
**Cerebro** is the cornerstone of backtrader because it serves as a central point for:

- Gathering all inputs (Data Feeds),
- Gathering actors (Stratgegies),
- Gathering critics (Analyzers)
- Execute the backtesting/or live data feeding/trading
- Returning the results

In [ ]:
# Initialize Engine
cerebro = bt.Cerebro()

# Initialize Broker Configs
cerebro.broker.setcash(100000.0)  # Starting with 1 Lakh cash
cerebro.broker.setcommission(0.0003)  # 0.03% Brokerage

# Feed Data
data0 = bt.feeds.PandasData(dataname=data, tz=pytz.timezone('Asia/Kolkata'))
cerebro.adddata(data0)


class BaseStrategy(bt.Strategy):
    def __init__(self):
        super().__init__()
        self.order = None

    def log(self, txt, dt=None):
        """Logging function for strategy"""
        dt = dt or self.data.datetime.datetime(0)
        print(f'{dt.isoformat()}: {txt}')


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        if order.status in [order.Completed]:
            if order.isbuy():
                # self.log(f'BUY  Executed, {order.executed.price:.2f}')
                pass
            elif order.issell():
                # self.log(f'SELL Executed, {order.executed.price:.2f}')
                pass
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(f'Order failed({order.getstatusname()})')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log("Trade Summary, gross: %.2f, duration: %d\n" % (trade.pnl, trade.barlen))


class MarubozuStrategy(BaseStrategy):
    params = dict(
        rrr=2,  # risk reward ratio
    )

    def __init__(self):
        super().__init__()
        self.orefs = []

        # Marubozu indicator
        self.marubozu = bt.talib.CDLMARUBOZU(self.data.open, self.data.high, self.data.low, self.data.close)

    def notify_order(self, order):
        if not order.alive() and order.ref in self.orefs:
            self.orefs.remove(order.ref)

    def next(self):
        if self.orefs:
            return  # pending orders do nothing

        # Check if we are in the market
        if not self.position:
            # We are not in the market, look for a signal to OPEN trades
            if self.marubozu[0] == 100:
                # strong buy momentum, open long position
                stoploss = self.data.low[0]  # Low of marubozu will act as stoploss
                
                # Add minimum risk validation
                risk = self.data.close[0] - stoploss
                if risk <= 0:
                    return  # Skip if risk is invalid
                
                target = (self.params.rrr * risk) + self.data[0]
                self.log(f'Long : %.2f, Target: %.2f, Stoploss: %.2f' % (self.data[0], target, stoploss))
                os = self.buy_bracket(price=self.data[0], limitprice=target, stopprice=stoploss)
                self.orefs = [o.ref for o in os]
            elif self.marubozu[0] == -100:
                # strong sell momentum, open short position
                stoploss = self.data.high[0]  # High of marubozu will act as stoploss
                
                # Add minimum risk validation
                risk = stoploss - self.data.close[0]
                if risk <= 0:
                    return  # Skip if risk is invalid
                
                target = self.data[0] - (self.params.rrr * risk)
                self.log(f'Short: %.2f, Target: %.2f, Stoploss: %.2f' % (self.data[0], target, stoploss))
                os = self.sell_bracket(price=self.data[0], limitprice=target, stopprice=stoploss)
                self.orefs = [o.ref for o in os]


# Add Strategy
cerebro.addstrategy(MarubozuStrategy)

# Add analyzers for better performance tracking
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe")
cerebro.addanalyzer(bt.analyzers.DrawDown, _name="dd")

# Add position sizer for better risk management
cerebro.addsizer(bt.sizers.PercentSizer, percents=10)  # Risk 10% of capital per trade

# Backtest
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
result = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

## Understanding BackTrader Strategy

### Base Strategy

In [44]:
class BaseStrategy(bt.Strategy):
    def __init__(self):
        super().__init__()
        self.order = None

    def log(self, txt, dt=None):
        """Logging function for strategy"""
        dt = dt or self.data.datetime.datetime(0)
        print(f'{dt.isoformat()}: {txt}')


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log("BUY EXECUTED, price: %.2f, commision: %.2f" % (order.executed.price, order.executed.comm))
                pass
            elif order.issell():
                # self.log(f'SELL Executed, {order.executed.price:.2f}')
                pass
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(f'Order failed({order.getstatusname()})')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log("Trade Summary, gross: %.2f, net: %.2f, duration: %d\n" % (trade.pnl, trade.pnlcomm, trade.barlen))

### Log Close Price Strategy

In [30]:
class LogCloseStrategy(BaseStrategy):
    def next(self):
        # Simply log the closing price
        self.log('Close, %.2f' % self.data.close[0])

### Simple Buy Strategy

The Simple Buy Strategy buys a stock when its price falls for three consecutive sessions.

In [ ]:
class SimpleBuyStrategy(BaseStrategy):
    def next(self):
        self.log('Close, %.2f' % self.data.close[0])

        if self.data.close[0] < self.data.close[-1]:
            if self.data.close[-1] < self.data.close[-2]:
                # price has been falling 3 sessions in a row
                self.log('BUY Triggered, %.2f' % self.data.close[0])
                self.buy()

### Simple Trade Strategy

The Simple Trade Strategy buys a stock when its price falls for three consecutive sessions and sells it after holding for n sessions.

In [ ]:
class SimpleTradeStrategy(BaseStrategy):
    params = (('exitbars', 5),)

    def next(self):
        # self.log('Close, %.2f' % self.data.close[0])

        if not self.position:
            if self.data.close[0] < self.data.close[-1]:
                if self.data.close[-1] < self.data.close[-2]:
                    # price has been falling 3 sessions in a row
                    self.buy()
        else:
            if len(self) >= (self.bar_executed + self.params.exitbars):
                self.sell()

## Analyze Trading Strategy

In [32]:
def display_trade_analysis(stats):
    """
    Displays a summary of trade analysis based on Backtrader's Analyzer results.

    This function prints out key statistics such as the number of open trades, closed trades, win/loss ratio,
    longest winning/losing streak, and overall net profit/loss. It also calculates and displays the strike rate
    (percentage of won trades out of total closed trades).

    :param stats: Backtrader's TradeAnalyzer object that holds trade performance metrics.
    """
    try:
        # Retrieve key metrics from the trade statistics
        total_open = stats.total.open       # Number of trades still open
        total_closed = stats.total.closed   # Number of trades that have been closed
        total_won = stats.won.total         # Total number of winning trades
        total_lost = stats.lost.total       # Total number of losing trades
        win_streak = stats.streak.won.longest   # Longest winning streak
        lost_streak = stats.streak.lost.longest # Longest losing streak
        pnl_net = round(stats.pnl.net.total, 2) # Net profit/loss, rounded to 2 decimal places

        # Calculate strike rate (percentage of closed trades that were winners)
        strike_rate = round((total_won / total_closed) * 100, 2) if total_closed > 0 else 0

        # Define headers and values for display
        headers_1 = ["Total Open", "Total Closed", "Total Won", "Total Lost"]
        headers_2 = ["Strike Rate(%)", "Win Streak", "Loss Streak", "Net PnL"]
        row_1 = [total_open, total_closed, total_won, total_lost]
        row_2 = [strike_rate, win_streak, lost_streak, pnl_net]

        # Determine the longest header length for formatting consistency
        max_header_length = max(len(headers_1), len(headers_2))

        # Prepare rows for display
        rows_to_display = [headers_1, row_1, headers_2, row_2]

        # Define the formatting for the display (20 characters width for each column)
        row_format = "{:<20}" * (max_header_length + 1)

        # Display the results
        print("\nTrade Analysis Results:")
        print("-" * 100)
        for row in rows_to_display:
            print(row_format.format("", *row))
        print("-" * 100)

    except (KeyError, AttributeError) as e:
        # Handle cases where no trade data is available or stats fields are missing
        print("No trade analysis available. It seems no trades have been taken or the stats are incomplete.")
        print(f"Error details: {e}")


### Display Analysis Result

In [ ]:
# Enhanced analysis display with additional metrics
def display_enhanced_analysis(result):
    """Display comprehensive backtesting analysis"""
    strat = result[0]
    
    # Trade Analysis
    ta = strat.analyzers.ta.get_analysis()
    display_trade_analysis(ta)
    
    # Sharpe Ratio
    sharpe = strat.analyzers.sharpe.get_analysis()
    if 'sharperatio' in sharpe and sharpe['sharperatio'] is not None:
        print(f"\nSharpe Ratio: {sharpe['sharperatio']:.4f}")
    
    # Drawdown Analysis
    dd = strat.analyzers.dd.get_analysis()
    print(f"Max Drawdown: {dd['max']['drawdown']:.2f}%")
    print(f"Max Drawdown Duration: {dd['max']['len']} periods")

# Display comprehensive analysis
display_enhanced_analysis(result)

## Implementing Practical Trade Strategies

### Simple Moving Average Crossover Strategy

Traders often use moving averages to identify trends and potential entry/exit points. A basic moving average strategy might involve buying when the price moves above a moving average and selling when it falls below


A moving average crossover system is an improvisation over the plain vanilla moving average system. Instead of the usual single moving average in a MA crossover system, the trader combines two moving averages.

The entry and exit rules for the crossover system is as stated below:

1. Buy (fresh long) when the short term moving averages turns greater than the long term moving average. Stay in the trade as long as this condition is satisfied

2. Exit the long position (square off) when the short term moving average turns lesser than the longer-term moving average

In [35]:
class SmaCross(BaseStrategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=5,  # period for the fast moving average
        pslow=10   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.params.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.params.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.log(f'Buy : %.2f' % (self.data[0]))
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.log(f'Sell: %.2f' % (self.data[0]))
            self.close()  # close long position

### Exponential Moving Average Crossover Strategy

An exponential moving average (EMA) scales the data according to its newness. Recent data gets the maximum weightage, and the oldest gets the least weightage.

In a crossover system, the price chart is overlaid with two EMAs. The shorter EMA is faster to react, while the longer EMA is slower to react.

The outlook turns bullish when the faster EMA crosses and is above the slower EMA. Hence one should look at buying the stock. The trade lasts upto a point where the faster EMA starts going below, the slower EMA

In [36]:
class EmaCross(BaseStrategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=1,  # period for the fast moving average
        pslow=10   # period for the slow moving average
    )

    def __init__(self):
        ema1 = bt.ind.EMA(period=self.params.pfast)  # fast moving average
        ema2 = bt.ind.EMA(period=self.params.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(ema1, ema2)  # crossover signal

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.log(f'Buy : %.2f' % (self.data[0]))
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.log(f'Sell: %.2f' % (self.data[0]))
            self.close()  # close long position

### Relative strength Index Strategy
Relative strength Index or just RSI is the most popular leading indicator, which gives out the strongest signals during the periods of sideways and non-trending ranges.

RSI is a momentum oscillator which oscillates between 0 and 100 level.

A value between 0 and 30 is considered oversold. Hence the trader should look at buying opportunities.

A value between 70 and 100 is considered overbought. Hence the trader should look at selling opportunities.

In [37]:
class RSI(BaseStrategy):
    # List of parameters which are configurable for the strategy
    params = dict(
        period=14,            # Period for the RSI calculation
        rsi_overbought=70,    # Upper threshold for RSI (overbought)
        rsi_oversold=30       # Lower threshold for RSI (oversold)
    )

    def __init__(self):
        # Initialize the RSI indicator
        self.rsi = bt.ind.RSI(period=self.params.period)

    def next(self):
        # self.log(self.rsi[0])
        if not self.position:
            if self.rsi[0] < self.params.rsi_oversold:
                self.log(f'Buy : %.2f' % (self.data[0]))
                self.buy()  # enter long
        elif self.rsi[0] > self.params.rsi_overbought:
            self.log(f'Sell: %.2f' % (self.data[0]))
            self.close()  # close long position

## Strategy with Target and StopLoss

### MaruBozu Strategy

Marubozu is a candlestick with no upper and lower shadow.

- A bullish marubozu indicates bullishness.
    - Buy around the closing price of a bullish marubozu
    - Keep the low of the marubozu as the stoploss
- A bearish marubozu indicates bearishness.
    - Sell around the closing price of a bearish marubozu
    - Keep the high of the marubozu as the stoploss

In [ ]:
class MarubozuStrategy(BaseStrategy):
    params = dict(
        rrr=2,  # risk reward ratio
    )

    def __init__(self):
        super().__init__()
        self.orefs = []

        # Marubozu indicator
        self.marubozu = bt.talib.CDLMARUBOZU(self.data.open, self.data.high, self.data.low, self.data.close)

    def notify_order(self, order):
        if not order.alive() and order.ref in self.orefs:
            self.orefs.remove(order.ref)

    def next(self):
        if self.orefs:
            return  # pending orders do nothing

        # Check if we are in the market
        if not self.position:
            # We are not in the market, look for a signal to OPEN trades
            if self.marubozu[0] == 100:
                # strong buy momentum, open long position
                stoploss = self.data.low[0]  # Low of marubozu will act as stoploss
                target = (self.params.rrr * (self.data.close[0] - stoploss)) + self.data[0]
                self.log(f'Long : %.2f, Target: %.2f, Stoploss: %.2f' % (self.data[0], target, stoploss))
                os = self.buy_bracket(price=self.data[0], limitprice=target, stopprice=stoploss)
                self.orefs = [o.ref for o in os]
            elif self.marubozu[0] == -100:
                # strong sell momentum, open short position
                stoploss = self.data.high[0]  # Fixed: was self.data_high[0], should be self.data.high[0]
                target = self.data[0] - (self.params.rrr * (stoploss - self.data[0]))
                self.log(f'Short: %.2f, Target: %.2f, Stoploss: %.2f' % (self.data[0], target, stoploss))
                os = self.sell_bracket(price=self.data[0], limitprice=target, stopprice=stoploss)
                self.orefs = [o.ref for o in os]

## Appendix

### Logging DateTime with Timezone Support in Backtrader
When analyzing intraday trades, having accurate timestamps with timezone support can be crucial for understanding when trades occur in relation to market hours. Backtrader supports timezone handling, allowing you to log the exact datetime of each event during strategy execution.



In [38]:
import pytz

class BaseStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        """Logging function for strategy"""
        dt = dt or self.data.datetime.datetime(0)
        print(f'{dt.isoformat()}: {txt}')

# When feeding your data into Backtrader,
# you can specify the timezone using the tz parameter.
data0 = bt.feeds.PandasData(dataname=data, tz=pytz.timezone('Asia/Kolkata'))

### Simulating Broker Commision

Within the regular cerebro creation/set-up process, just add a call to setcommission over the broker member attribute.

In [39]:
cerebro.broker.setcommission(0.0003)  # 0.03% Brokerage

To Log the commision related info use following:

In [ ]:
# Example usage within a strategy's notify_trade and notify_order methods:
# self.log("Trade Summary, gross: %.2f, net: %.2f, duration: %d\n" % (trade.pnl, trade.pnlcomm, trade.barlen))
# self.log("BUY EXECUTED, price: %.2f, commision: %.2f" % (order.executed.price, order.executed.comm))

### Customize Order Size

## Stock Screening for Trading Opportunities

The following functions demonstrate how to use this framework for stock screening and signal detection for next-day trading decisions.

In [ ]:
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

def screen_stocks_for_signals(symbols_list, strategy_type="marubozu", period="1mo", interval="1d"):
    """
    Screen multiple stocks for trading signals
    
    Args:
        symbols_list: List of stock symbols to screen
        strategy_type: Type of strategy ('marubozu', 'rsi', 'sma_cross', 'ema_cross')
        period: Data period to fetch
        interval: Data interval
        
    Returns:
        List of dictionaries with stock signals and details
    """
    signals = []
    
    for symbol in symbols_list:
        try:
            # Get stock data
            data = get_instrument_data(symbol, period, interval)
            
            if data.empty:
                continue
                
            # Apply strategy logic based on type
            if strategy_type == "marubozu":
                signal = detect_marubozu_signal(data, symbol)
            elif strategy_type == "rsi":
                signal = detect_rsi_signal(data, symbol)
            elif strategy_type == "sma_cross":
                signal = detect_sma_cross_signal(data, symbol)
            elif strategy_type == "ema_cross":
                signal = detect_ema_cross_signal(data, symbol)
            else:
                continue
                
            if signal:
                signals.append(signal)
                
        except Exception as e:
            print(f"Error processing {symbol}: {str(e)}")
            continue
            
    return signals

def detect_marubozu_signal(data, symbol):
    """Detect Marubozu pattern signal"""
    try:
        # Calculate Marubozu using TA-Lib
        marubozu = talib.CDLMARUBOZU(data['Open'].values, data['High'].values, 
                                   data['Low'].values, data['Close'].values)
        
        # Check latest signal
        latest_signal = marubozu[-1]
        current_price = data['Close'].iloc[-1]
        current_date = data.index[-1]
        
        if latest_signal == 100:  # Bullish Marubozu
            stoploss = data['Low'].iloc[-1]
            risk = current_price - stoploss
            target = current_price + (2 * risk)  # 1:2 risk-reward
            
            return {
                'symbol': symbol,
                'signal': 'BUY',
                'strategy': 'Marubozu',
                'current_price': round(current_price, 2),
                'target': round(target, 2),
                'stoploss': round(stoploss, 2),
                'risk_reward': '1:2',
                'date': current_date.strftime('%Y-%m-%d'),
                'confidence': 'High'
            }
            
        elif latest_signal == -100:  # Bearish Marubozu
            stoploss = data['High'].iloc[-1]
            risk = stoploss - current_price
            target = current_price - (2 * risk)  # 1:2 risk-reward
            
            return {
                'symbol': symbol,
                'signal': 'SELL',
                'strategy': 'Marubozu',
                'current_price': round(current_price, 2),
                'target': round(target, 2),
                'stoploss': round(stoploss, 2),
                'risk_reward': '1:2',
                'date': current_date.strftime('%Y-%m-%d'),
                'confidence': 'High'
            }
    except:
        pass
    
    return None

def detect_rsi_signal(data, symbol, rsi_period=14):
    """Detect RSI-based signals"""
    try:
        # Calculate RSI
        rsi = talib.RSI(data['Close'].values, timeperiod=rsi_period)
        
        current_rsi = rsi[-1]
        current_price = data['Close'].iloc[-1]
        current_date = data.index[-1]
        
        if current_rsi < 30:  # Oversold - Buy signal
            return {
                'symbol': symbol,
                'signal': 'BUY',
                'strategy': 'RSI Oversold',
                'current_price': round(current_price, 2),
                'rsi_value': round(current_rsi, 2),
                'date': current_date.strftime('%Y-%m-%d'),
                'confidence': 'Medium'
            }
            
        elif current_rsi > 70:  # Overbought - Sell signal
            return {
                'symbol': symbol,
                'signal': 'SELL',
                'strategy': 'RSI Overbought',
                'current_price': round(current_price, 2),
                'rsi_value': round(current_rsi, 2),
                'date': current_date.strftime('%Y-%m-%d'),
                'confidence': 'Medium'
            }
    except:
        pass
        
    return None

def detect_sma_cross_signal(data, symbol, fast_period=5, slow_period=10):
    """Detect SMA crossover signals"""
    try:
        # Calculate SMAs
        sma_fast = talib.SMA(data['Close'].values, timeperiod=fast_period)
        sma_slow = talib.SMA(data['Close'].values, timeperiod=slow_period)
        
        # Check for crossover
        if len(sma_fast) < 2 or len(sma_slow) < 2:
            return None
            
        current_price = data['Close'].iloc[-1]
        current_date = data.index[-1]
        
        # Bullish crossover: fast SMA crosses above slow SMA
        if sma_fast[-1] > sma_slow[-1] and sma_fast[-2] <= sma_slow[-2]:
            return {
                'symbol': symbol,
                'signal': 'BUY',
                'strategy': f'SMA Cross ({fast_period}/{slow_period})',
                'current_price': round(current_price, 2),
                'fast_sma': round(sma_fast[-1], 2),
                'slow_sma': round(sma_slow[-1], 2),
                'date': current_date.strftime('%Y-%m-%d'),
                'confidence': 'Medium'
            }
            
        # Bearish crossover: fast SMA crosses below slow SMA
        elif sma_fast[-1] < sma_slow[-1] and sma_fast[-2] >= sma_slow[-2]:
            return {
                'symbol': symbol,
                'signal': 'SELL',
                'strategy': f'SMA Cross ({fast_period}/{slow_period})',
                'current_price': round(current_price, 2),
                'fast_sma': round(sma_fast[-1], 2),
                'slow_sma': round(sma_slow[-1], 2),
                'date': current_date.strftime('%Y-%m-%d'),
                'confidence': 'Medium'
            }
    except:
        pass
        
    return None

def detect_ema_cross_signal(data, symbol, fast_period=5, slow_period=20):
    """Detect EMA crossover signals"""
    try:
        # Calculate EMAs
        ema_fast = talib.EMA(data['Close'].values, timeperiod=fast_period)
        ema_slow = talib.EMA(data['Close'].values, timeperiod=slow_period)
        
        # Check for crossover
        if len(ema_fast) < 2 or len(ema_slow) < 2:
            return None
            
        current_price = data['Close'].iloc[-1]
        current_date = data.index[-1]
        
        # Bullish crossover: fast EMA crosses above slow EMA
        if ema_fast[-1] > ema_slow[-1] and ema_fast[-2] <= ema_slow[-2]:
            return {
                'symbol': symbol,
                'signal': 'BUY',
                'strategy': f'EMA Cross ({fast_period}/{slow_period})',
                'current_price': round(current_price, 2),
                'fast_ema': round(ema_fast[-1], 2),
                'slow_ema': round(ema_slow[-1], 2),
                'date': current_date.strftime('%Y-%m-%d'),
                'confidence': 'Medium'
            }
            
        # Bearish crossover: fast EMA crosses below slow EMA
        elif ema_fast[-1] < ema_slow[-1] and ema_fast[-2] >= ema_slow[-2]:
            return {
                'symbol': symbol,
                'signal': 'SELL',
                'strategy': f'EMA Cross ({fast_period}/{slow_period})',
                'current_price': round(current_price, 2),
                'fast_ema': round(ema_fast[-1], 2),
                'slow_ema': round(ema_slow[-1], 2),
                'date': current_date.strftime('%Y-%m-%d'),
                'confidence': 'Medium'
            }
    except:
        pass
        
    return None

In [ ]:
def daily_stock_screener():
    """
    Daily workflow to screen stocks for trading opportunities
    Run this function every evening to get next day's trading candidates
    """
    
    # Indian Stock Universe - Add more symbols as needed
    nifty_50_symbols = [
        "RELIANCE.NS", "TCS.NS", "HDFCBANK.NS", "INFY.NS", "ICICIBANK.NS",
        "HINDUNILVR.NS", "ITC.NS", "SBIN.NS", "BHARTIARTL.NS", "KOTAKBANK.NS",
        "LT.NS", "ASIANPAINT.NS", "AXISBANK.NS", "MARUTI.NS", "SUNPHARMA.NS",
        "TITAN.NS", "ULTRACEMCO.NS", "WIPRO.NS", "NESTLEIND.NS", "POWERGRID.NS"
    ]
    
    print(f"🔍 Daily Stock Screening - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 80)
    
    # Screen for different strategies
    strategies = ["marubozu", "rsi", "sma_cross", "ema_cross"]
    all_signals = {}
    
    for strategy in strategies:
        print(f"\n📊 Screening for {strategy.upper()} signals...")
        signals = screen_stocks_for_signals(nifty_50_symbols, strategy, period="3mo", interval="1d")
        
        if signals:
            all_signals[strategy] = signals
            print(f"✅ Found {len(signals)} {strategy} signals")
        else:
            print(f"❌ No {strategy} signals found")
    
    return all_signals

def display_signals_summary(all_signals):
    """Display formatted summary of all signals"""
    
    if not all_signals:
        print("📭 No trading signals found today.")
        return
    
    print(f"\n📈 TRADING OPPORTUNITIES FOR NEXT SESSION")
    print("=" * 80)
    
    # Combine all signals
    combined_signals = []
    for strategy, signals in all_signals.items():
        combined_signals.extend(signals)
    
    # Separate by signal type
    buy_signals = [s for s in combined_signals if s['signal'] == 'BUY']
    sell_signals = [s for s in combined_signals if s['signal'] == 'SELL']
    
    # Display BUY signals
    if buy_signals:
        print(f"\n🟢 BUY OPPORTUNITIES ({len(buy_signals)} stocks):")
        print("-" * 80)
        print(f"{'SYMBOL':<15} {'STRATEGY':<20} {'PRICE':<10} {'TARGET':<10} {'SL':<10} {'CONF':<8}")
        print("-" * 80)
        
        for signal in buy_signals:
            target = signal.get('target', 'N/A')
            stoploss = signal.get('stoploss', 'N/A')
            confidence = signal.get('confidence', 'N/A')
            
            print(f"{signal['symbol']:<15} {signal['strategy']:<20} {signal['current_price']:<10} "
                  f"{target:<10} {stoploss:<10} {confidence:<8}")
    
    # Display SELL signals  
    if sell_signals:
        print(f"\n🔴 SELL OPPORTUNITIES ({len(sell_signals)} stocks):")
        print("-" * 80)
        print(f"{'SYMBOL':<15} {'STRATEGY':<20} {'PRICE':<10} {'TARGET':<10} {'SL':<10} {'CONF':<8}")
        print("-" * 80)
        
        for signal in sell_signals:
            target = signal.get('target', 'N/A')
            stoploss = signal.get('stoploss', 'N/A')
            confidence = signal.get('confidence', 'N/A')
            
            print(f"{signal['symbol']:<15} {signal['strategy']:<20} {signal['current_price']:<10} "
                  f"{target:<10} {stoploss:<10} {confidence:<8}")
    
    print("\n" + "=" * 80)
    print("💡 NEXT STEPS:")
    print("1. Review signals and do your own analysis")
    print("2. Check market sentiment and news")
    print("3. Set appropriate position sizes")
    print("4. Place orders with proper risk management")
    print("=" * 80)

def save_signals_to_csv(all_signals, filename=None):
    """Save signals to CSV file for record keeping"""
    
    if not all_signals:
        return None
        
    if filename is None:
        filename = f"trading_signals_{datetime.now().strftime('%Y%m%d')}.csv"
    
    # Combine all signals
    combined_signals = []
    for strategy, signals in all_signals.items():
        combined_signals.extend(signals)
    
    # Convert to DataFrame
    df = pd.DataFrame(combined_signals)
    df.to_csv(filename, index=False)
    print(f"💾 Signals saved to: {filename}")
    return filename

### Example: Daily Screening Workflow

Run the following cell every evening to get next day's trading opportunities:

In [ ]:
# Daily screening example
signals_found = daily_stock_screener()

# Display results
display_signals_summary(signals_found)

# Save to CSV for records
if signals_found:
    save_signals_to_csv(signals_found)

In [46]:
cerebro.addsizer(bt.sizers.AllInSizerInt)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)